# Cabinet Start-Up Procedure

1. Turn on cabinet power in back of cabinet
2. Turn on sensor power (24 Volts). The sensor power inputs are labeled.
3. Compile code to AMDC
4. Turn on power to DC bus of cabinet.
5. Run Jupyter Notebook. 

Import neccessary modules

In [6]:
amdc.disconnect()

In [2]:
from Cabinet_CMD import cmd

import sys
sys.path.append(r'..\AMDC-Firmware\scripts')
from AMDC import AMDC
amdc = AMDC(port = 'COM13', cmdEcho = True, printOutput=True)
Vdc = 40
cmd = cmd(Vdc, amdc, debug = False) 

	> cabinet setup 40.00000
['cabinet setup 40.00000', 'SUCCESS']
	> cabinet init_cb
['cabinet init_cbENC:\tSetting pulses per rev bits = 10...', '', 'SUCCESS']


In [5]:
cmd.disable_pwm()

	> hw pwm off
['hw pwm off', 'SUCCESS']


In [4]:
cmd.bim_init()
cmd.bim_set_Vdc(40)
cmd.bim_set_id(10)
cmd.enable_pwm()
cmd.bim_enable_ctrl()



	> bim init
['bim initENC:\tSetting pulses per rev bits = 10...', '', 'SUCCESS']
	> bim set_vdc 40.000000
['bim set_vdc 40.000000', 'SUCCESS']
	> bim set_id 10.000000
['bim set_id 10.000000', 'SUCCESS']
	> hw pwm on
['hw pwm on', 'SUCCESS']
	> bim enable_ctrl
['bim enable_ctrl', 'SUCCESS']


In [ ]:
PI = 3.1415926
#cmd.bim_set_theta_offset(-PI/12)
#logger.clear_all()

cmd.bim_enable_ob()
cmd.bim_enable_vctrl()
cmd.bim_set_id(13.5)
#logger.start()
#cmd.bim_set_Fxy(0,3)
cmd.bim_set_deltaxy(0, 0)
cmd.bim_enable_levctrl()
#time.sleep(6)
#logger.stop()
#cmd.bim_set_id(10)
#data = logger.dump(file = 'bim_data_start.csv')

In [ ]:
#import matplotlib
import time

import pathlib as pl
import sys
repo_dir = '..'                                      # CHANGE THIS TO YOUR REPO DIRECTORY
repo_dir = pl.Path(repo_dir)

from AMDC_Logger import AMDC_Logger, find_mapfile
mapfile_path = find_mapfile(repo_dir)
logger = AMDC_Logger(AMDC = amdc, mapfile = mapfile_path)
logger.sync()

# View which variables are logged
logger.info()
#amdc.connect()
cmd.enable_log()

In [ ]:
logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm wrm wsl theta_e we Itq_d_ref Itq_q_ref Itq_d Itq_q Te', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm theta_e wsl', samples_per_sec = 1000, var_type = 'double')
#logger.register('wsl wrm theta_e theta_rm Itq_d_ref Itq_q_ref Itq_d Itq_q va1_ref vb1_ref vc1_ref va2_ref vb2_ref vc2_ref Is1_x Is1_y Is1_x_ref Is1_y_ref delta_x delta_y', samples_per_sec = 1000, var_type = 'double')
#logger.register('Is1_x Is1_y Is1_x_ref Is1_y_ref delta_x delta_y', samples_per_sec = 1000, var_type = 'double')
#logger.register('delta_x delta_y delta_x_ref delta_y_ref delta_x_ref_lpf delta_y_ref_lpf Is1_x Is1_y Is1_x_ref Is1_y_ref', samples_per_sec = 1000, var_type = 'double')
#logger.register('wrm Itq_d_ref Itq_q_ref Itq_d Itq_q Is1_x Is1_y Is1_x_ref Is1_y_ref', samples_per_sec = 1000, var_type = 'double')
logger.register('delta_x delta_y F_x F_y Is1_x Is1_y Is1_x_ref Is1_y_ref Itq_d_ref Itq_q_ref Itq_d Itq_q', samples_per_sec = 10000, var_type = 'double')
#logger.register('delta_x delta_y', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()


In [ ]:
cmd.bim_set_id(0)
cmd.bim_set_ixy(0,0)
logger.clear_all()
cmd.bim_set_ixy(0,0)
###################   COLLECT DATA   ####################
# Clear the logger, then record data

logger.start()
cmd.bim_set_ixy(3,0)
#cmd.bim_set_ixy(6, 0)
#DATA IS BEING RECORDED
time.sleep(0.02)

logger.stop()

In [ ]:
cmd.bim_set_w(00*PI/30)


In [ ]:
f = 20
w = 2*PI*f
cmd.bim_set_injection_Fxy(w, 10)


In [ ]:
cmd.bim_disable_injection()

In [ ]:

for f in [75, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 700, 800, 900]:
    logger.stop()
    w = 2*PI*f

    cmd.bim_set_injection_Fxy(w, 5)
    logger.clear_all()
    logger.start()
#cmd.bim_set_ixy(3,0)
#cmd.bim_set_ixy(6, 0)
#DATA IS BEING RECORDED
    time.sleep(0.05)
    logger.stop()
    filename = 'bim_data_2000RPM_stiff_f' + str(f)+'.csv'
####################   DUMP DATA AND PLOT   ####################
    data = logger.dump(file = filename)
cmd.bim_disable_injection()
cmd.bim_set_w(100*PI/30)

In [ ]:
i = 2
vd_ref_chirp_gain     = 5000 * 1e-3; # [m]
vd_ref_chirp_freq_min = i; # [Hz]
vd_ref_chirp_freq_max = i; # [Hz]
vd_ref_chirp_period   = 0.1; # [sec]
amdc.cmd("inj const vq_ref add 0")
amdc.cmd("inj const vx_ref add 0")
amdc.cmd("inj const vy_ref add 0")
amdc.cmd("inj chirp vd_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))


In [ ]:

logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
#for i in [1, 2, 3, 5, 8, 10, 15, 20, 30, 40, 50, 60, 75, 90, 100]:
for i in [1, 2]:
   
    logger.stop()
    logger.unregister_all()
    logger.clear_all()
    sampling = 1000
    period = 1
    if i <= 10:
        sampling = 1000
        period = 1
    elif i<= 100:
        sampling = 1000
        period = 0.2

    else:
        sampling = 2500
        period = 0.1

    logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
    logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
    #logger.register('Itq_d vd_ref', samples_per_sec = sampling, var_type = 'double')
    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data
   
    logger.clear_all()
    vd_ref_chirp_gain     = 3000 * 1e-3; # [m]
    vd_ref_chirp_freq_min = i; # [Hz]
    vd_ref_chirp_freq_max = i; # [Hz]
    vd_ref_chirp_period   = period; # [sec]
    amdc.cmd("inj const vq_ref add 0")
    amdc.cmd("inj const vx_ref add 0")
    amdc.cmd("inj const vy_ref add 0")
    amdc.cmd("inj chirp vd_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'bim_data_id_vd'+ str(i)+'.csv'
    data = logger.dump(file = filename)

 

    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data

    logger.clear_all()
    amdc.cmd("inj const vd_ref add 0")
    amdc.cmd("inj const vq_ref add 0")
    amdc.cmd("inj const vy_ref add 0")
    amdc.cmd("inj chirp vx_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'bim_data_id_vx'+ str(i)+'.csv'
    data = logger.dump(file = filename)

In [ ]:
logger.clear_all()
logger.start()
#cmd.bim_set_ixy(3,0)
#cmd.bim_set_ixy(6, 0)
#DATA IS BEING RECORDED
time.sleep(0.1)
logger.stop()
####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'bim_data_3100RPM.csv')

In [ ]:
time.sleep(0.1)
logger.stop()

In [ ]:

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'bim_data_Dis.csv')

In [ ]:

#data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
#data.filter(regex="^wsl$").plot()
#data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
#data.filter(regex="^Itq_d_ref|Itq_d$").plot()
#data.filter(regex="^Itq_q_ref|Itq_q$").plot()
data.filter(regex="^Is1_x_ref|Is1_x$").plot()
data.filter(regex="^Is1_y_ref|Is1_y$").plot()

#data.filter(regex="^va1_ref|vb1_ref|vc1_ref$").plot()
#data.filter(regex="^va2_ref|vb2_ref|vc2_ref$").plot()
#data.filter(regex="^Te$").plot()
data.filter(regex="^F_x|F_y$").plot()
data.filter(regex="^delta_x|delta_y$").plot()
#data.filter(regex="^error_x|error_y$").plot()
#data.filter(regex="^delta_x|delta_y|delta_x_ref|delta_y_ref|delta_x_ref_lpf|delta_y_ref_lpf$").plot()

In [ ]:
#cmd.bim_enable_vctrl()

cmd.bim_set_w(1000*PI/30)
#cmd.bim_set_id(10)

In [ ]:
cmd.bim_set_ixy(0, 3)


In [ ]:
cmd.bim_enable_vctrl()
cmd.bim_set_id(6)


In [ ]:
cmd.bim_disable_levctrl()
cmd.bim_disable_vctrl()

In [ ]:
cmd.disable_pwm()


In [ ]:
amdc.disconnect()
#amdc.connect()

## Logging


In [ ]:
import glob, os
from pandas import *
import matplotlib.pyplot as plt
import numpy as np
import time
import pathlib as pl
import sys
repo_dir = '..'                                      # CHANGE THIS TO YOUR REPO DIRECTORY
repo_dir = pl.Path(repo_dir)


In [ ]:
for file in glob.glob("*.csv"):
    print(file)


In [ ]:
list_of_files = glob.glob('*.csv')
file_name = max(list_of_files, key=os.path.getctime)
print(file_name)

data = read_csv('bim_data_3000RPM_2021-10-05_H13-M42-S47.csv')

delta_x = data['delta_x'].tolist()
delta_y = data['delta_y'].tolist()

plt.plot(delta_x, delta_y)

plt.show

In [ ]:

from AMDC_Logger import AMDC_Logger, find_mapfile
mapfile_path = find_mapfile(repo_dir)
logger = AMDC_Logger(AMDC = amdc, mapfile = mapfile_path)
logger.sync()

# View which variables are logged
logger.info()
#amdc.connect()
cmd.c_loop_enable_log()

In [ ]:
logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm wrm wsl theta_e we Itq_d_ref Itq_q_ref Itq_d Itq_q Te', samples_per_sec = 1000, var_type = 'double')
logger.register('theta_rm theta_e wrm', samples_per_sec = 1000, var_type = 'double')



# View which variables are logged
logger.info()

###################   COLLECT DATA   ####################
# Clear the logger, then record data
logger.clear_all()
logger.start()


In [ ]:

#DATA IS BEING RECORDED
time.sleep(1)

logger.stop()


In [ ]:
####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'bim_data_pos.csv')


In [ ]:
data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
#data.filter(regex="^wsl$").plot()
data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
#data.filter(regex="^Itq_d_ref|Itq_d$").plot()
#data.filter(regex="^Itq_q_ref|Itq_q$").plot()
#data.filter(regex="^Te$").plot()


In [ ]:

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'bim_data_pos.csv')
#data


In [ ]:
data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
data.filter(regex="^wsl$").plot()
data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
data.filter(regex="^Itq_d_ref|Itq_d$").plot()
data.filter(regex="^Itq_q_ref|Itq_q$").plot()
data.filter(regex="^Te$").plot()


In [ ]:
logger.unregister_all()
logger.register('Iabc1_a Iabc1_b Iabc1_c', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
#time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data_Iabc.csv')
#data

data.filter(regex="^Iabc1_a|Iabc1_b|Iabc1_c$").plot()

In [ ]:
logger.unregister_all()
#logger.register('Itq_d_ref Itq_q_ref Itq_d Itq_q Is1_d_ref Is1_q_ref Is1_d Is1_q Is2_d_ref Is2_q_ref Is2_d Is2_q', samples_per_sec = 1000, var_type = 'double')
logger.register('va_ref vb_ref vc_ref', samples_per_sec = 1000, var_type = 'double')
# View which variables are logged
logger.info()

logger.clear_all()
logger.start()
time.sleep(0.5)
logger.stop()

data = logger.dump(file = 'bim_data_Idq.csv')
#data

data.filter(regex="^va_ref|vb_ref|vc_ref$").plot()
#data.filter(regex="^Itq_q_ref|Itq_q$").plot()
#data.filter(regex="^Te_ref$").plot()

In [ ]:
data.filter(regex="^Ib1_a|Ib1_b|Ib1_c$").plot()

In [ ]:
data.filter(regex="^Iabc2_a|Iabc2_b|Iabc2_c$").plot()

In [ ]:
data.filter(regex="^Iabc3_a|Iabc3_b|Iabc3_c$").plot()

In [ ]:
logger.unregister_all()
logger.register('vabc1_a vabc1_b vabc1_c', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^vabc1_a|vabc1_b|vabc1_c$").plot()

In [ ]:

data = logger.dump(file = 'bim_data.csv')
data


In [ ]:
data.filter(regex="^Itq_d_ref|Itq_q_ref|Itq_d$|Itq_q").plot()
#data.filter(regex="^Is1_d_ref|Is1_q_ref|Is1_d|Is1_q$").plot()
#data.filter(regex="^Is2_d_ref|Is2_q_ref|Is2_d|Is2_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('Is1_d_ref Is1_q_ref Is1_d Is1_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()


In [ ]:
#data = logger.dump(file = 'test_data.csv')
#data
data.filter(regex="^Is1_d_ref|Is1_q_ref|Is1_d|Is1_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('Is2_d_ref Is2_q_ref Is2_d Is2_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^Is2_d_ref|Is2_q_ref|Is2_d|Is2_q$").plot()

In [ ]:

logger.unregister_all()
logger.register('v_tq_d v_tq_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^v_tq_d|v_tq_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('we_tq', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^we_tq$").plot()